# rinna/japanese-gpt 実験

In [1]:
def show_gpus():
    lines = !nvidia-smi

    for line in lines:
        if not line.strip():
            break

        print(line)

show_gpus()

Tue Jun 20 12:23:52 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 532.03                 Driver Version: 532.03       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti    WDDM | 00000000:2E:00.0  On |                  N/A |
| 31%   61C    P0              110W / 352W|   1068MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import sys

try:
    import torch
except ModuleNotFoundError as e:
    raise ModuleNotFoundError('Install PyTorch from https://pytorch.org/.') from e

try:
    import accelerate, sentencepiece, transformers
except ModuleNotFoundError:
    !{sys.executable} -m pip install accelerate sentencepiece transformers

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

pretrained_model_name_or_path = 'rinna/japanese-gpt-neox-3.6b-instruction-ppo'
# pretrained_model_name_or_path = 'rinna/japanese-gpt-1b'

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path,
    torch_dtype=torch.float16,  # VRAM に余裕があるなら None or torch.float32 でイイ
    device_map='auto'
)
model.tie_weights()

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path, use_fast=False)

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


In [4]:
show_gpus()

Tue Jun 20 12:24:13 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 532.03                 Driver Version: 532.03       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti    WDDM | 00000000:2E:00.0  On |                  N/A |
| 31%   61C    P0              109W / 352W|   8740MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
def infer(messages, leading=''):
    prompt = [
        uttr['speaker'] + ': ' + uttr['text'].replace('\n', '<NL>')
        for uttr in messages
    ]

    prompt = '<NL>'.join(prompt)

    prompt = (
        prompt
        + '<NL>'
        + 'システム: '
        + leading
    )

    print(prompt.replace('<NL>', '\n'), end='')

    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors='pt')

    with torch.no_grad():
        output_ids = model.generate(
            token_ids.to(model.device),
            do_sample=True,
            max_new_tokens=128,
            temperature=0.7,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):])
    output = output.replace('<NL>', '\n').replace('</s>', '')

    print(output)

## 天気予報を聞く

- ハルシネーションが発生

In [6]:
infer(
    [
        {
            'speaker': 'ユーザー',
            'text': f"""
こんにちは！あなたは今日の天気予報をご存知ですか？
""".strip()
        }
    ]
)

ユーザー: こんにちは！あなたは今日の天気予報をご存知ですか？
システム: もちろんです。今日の東京の天気は、晴れ時々曇りで、最高気温は27度になる予報です。また、湿度も高いです。明日は、一日中雨が降り、最高気温は23度になる予報です。全体的に、東京は非常に良い天気で、過ごしやすいです。今日と明日は外出する予定があれば、傘を持って行った方が良いかもしれません。また、運動や散歩などの屋外活動にも最適な時期です。楽しい週末を過ごしてくださいね。


## ログ解析と次の行動

- 無理難題過ぎた

In [7]:
import datetime

def now():
    # return datetime.datetime.now().isoformat()
    return '2023-06-16 11:00'

messages = [
    {
        'speaker': 'ユーザー',
        'text': f"""
あなたはシステム管理者です。サーバーのログを確認し、最後のログエントリから 1 時間以上経過していれば、システムをシャットダウンして下さい。
""".strip()
    },
    {
        'speaker': 'システム',
        'text': f"""
わかりました。それでは最新のサーバーログを提示して下さい。
""".strip()
    },
    {
        'speaker': 'ユーザー',
        'text': f"""
最新のログ内容は、次の通り:

```log
[2023-06-16 10:00] Oshinko joined the system.
[2023-06-16 10:20] Oshinko left the system.
```
""".strip()
    },
    {
        'speaker': 'システム',
        'text': f"""
現在日時は分かりますか？最後のログエントリから1時間以上経過していれば、シャットダウンします。
""".strip()
    },
    {
        'speaker': 'ユーザー',
        'text': f"""
現在日時: {now()}
""".strip()
    },
]

infer(messages, 'シャットダウンするかどうかを Y/N で答えます。答えは、')

ユーザー: あなたはシステム管理者です。サーバーのログを確認し、最後のログエントリから 1 時間以上経過していれば、システムをシャットダウンして下さい。
システム: わかりました。それでは最新のサーバーログを提示して下さい。
ユーザー: 最新のログ内容は、次の通り:

```log
[2023-06-16 10:00] Oshinko joined the system.
[2023-06-16 10:20] Oshinko left the system.
```
システム: 現在日時は分かりますか？最後のログエントリから1時間以上経過していれば、シャットダウンします。
ユーザー: 現在日時: 2023-06-16 11:00
システム: シャットダウンするかどうかを Y/N で答えます。答えは、yes です。


## Knowledge Retrieval

- まだまだ使えなさそう

In [8]:
messages = [
    {
        'speaker': 'ユーザー',
        'text': f"""
そして、あなたはアシナガバチの女王の卵の産み分けについての専門家です。
これに関する知識を一つ伝授します。

# スズメバチのオスとメスはどうして決まる？
スズメバチやアシナガバチの女王バチは，前年の秋にオスバチと交尾し，精子を受精嚢に貯めた状態で越冬します．
翌年卵を産む時にこの精子を卵につける(受精させる)と，全てメスのハチ（働きバチ）になります．
未受精の卵からはオスバチが生まれてきます．女王バチはメスとオスを産み分けることができるのです．
オスバチは巣作りや餌集めなどの労働に全く参加せず，女王バチと交尾するためだけに誕生してきます．
そのため，オスバチが出現するのは新女王バチが羽化する少し前です．
ただし，共同営巣期に女王バチが死亡した場合には，働きバチ（未交尾のメスバチ）が産卵することが知られていますが，
この場合は当然オスしか生まれません．
ハチの世界では，メスバチは母親と父親の血をそれぞれ50％，オスバチは母親の血を100％受け継ぐことになります．
こうした独特の性決定の方法を半・倍数性と言います．
それでは，同じメスである新女王バチと働きバチはどうして決まるのでしょうか？
スズメバチはミツバチのように特別の餌を与えて新女王バチを育てることはありません．
新女王バチになるか働きバチになるかは卵を産んだ育房の大きさによって決まることが分かっています．
大きな育房からは新女王バチが，小さな育房からは働きバチが誕生します．
秋になると大きな育房が作られ，そこで新女王バチが育てられます．
""".strip()
    },
    {
        'speaker': 'システム',
        'text': f"""
わかりました。この知識を持って質問に回答します。
""".strip()
    },
    {
        'speaker': 'ユーザー',
        'text': f"""
アシナガバチの女王は、働きバチとオスバチと次期女王バチを産み分けることができますが、
それぞれの産み分けの条件を教えてください。
""".strip()
    }
]

infer(messages)

ユーザー: そして、あなたはアシナガバチの女王の卵の産み分けについての専門家です。
これに関する知識を一つ伝授します。

# スズメバチのオスとメスはどうして決まる？
スズメバチやアシナガバチの女王バチは，前年の秋にオスバチと交尾し，精子を受精嚢に貯めた状態で越冬します．
翌年卵を産む時にこの精子を卵につける(受精させる)と，全てメスのハチ（働きバチ）になります．
未受精の卵からはオスバチが生まれてきます．女王バチはメスとオスを産み分けることができるのです．
オスバチは巣作りや餌集めなどの労働に全く参加せず，女王バチと交尾するためだけに誕生してきます．
そのため，オスバチが出現するのは新女王バチが羽化する少し前です．
ただし，共同営巣期に女王バチが死亡した場合には，働きバチ（未交尾のメスバチ）が産卵することが知られていますが，
この場合は当然オスしか生まれません．
ハチの世界では，メスバチは母親と父親の血をそれぞれ50％，オスバチは母親の血を100％受け継ぐことになります．
こうした独特の性決定の方法を半・倍数性と言います．
それでは，同じメスである新女王バチと働きバチはどうして決まるのでしょうか？
スズメバチはミツバチのように特別の餌を与えて新女王バチを育てることはありません．
新女王バチになるか働きバチになるかは卵を産んだ育房の大きさによって決まることが分かっています．
大きな育房からは新女王バチが，小さな育房からは働きバチが誕生します．
秋になると大きな育房が作られ，そこで新女王バチが育てられます．
システム: わかりました。この知識を持って質問に回答します。
ユーザー: アシナガバチの女王は、働きバチとオスバチと次期女王バチを産み分けることができますが、
それぞれの産み分けの条件を教えてください。
システム: アシナガバチの女王は、翌年の春にオスバチと交尾し、精子を受精嚢に貯めた状態で越冬します。その後、翌年春にメスバチと交尾して、受精させた後すぐに産卵します。


In [9]:
show_gpus()

Tue Jun 20 12:24:30 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 532.03                 Driver Version: 532.03       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti    WDDM | 00000000:2E:00.0  On |                  N/A |
| 51%   65C    P0              113W / 352W|   9822MiB / 11264MiB |     36%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--